In [8]:
#!/usr/bin/env python3
from Crypto.Hash import SHA256
from Crypto.Cipher import AES
import signal, random
random = random.SystemRandom()

q = 0x3a05ce0b044dade60c9a52fb6a3035fc9117b307ca21ae1b6577fef7acd651c1f1c9c06a644fd82955694af6cd4e88f540010f2e8fdf037c769135dbe29bf16a154b62e614bb441f318a82ccd1e493ffa565e5ffd5a708251a50d145f3159a5

def enc(a):
    f = {str: str.encode, int: int.__str__}.get(type(a))
    return enc(f(a)) if f else a

def H(*args):
    data = b'\0'.join(map(enc, args))
    return SHA256.new(data).digest()

def F(h, x):
    return pow(h, x, q)

################################################################

password = random.randrange(10**6)

def go(publicB,verB):
    generator = int(H(password).hex(), 16)

    private_A = 40*random.randrange(2**999)
    publicA = F(generator, private_A)
    print(f'{publicA = :#x}')

    if not 1 < publicB < q:
        exit('nope')

    shared = F(publicB, private_A)

    verA = F(generator, shared**3)
    print(f'{verA = :#x}')

    if verB == F(generator, shared**5):
        key = H(password, shared)
        flag = "this is a test"
        aes = AES.new(key, AES.MODE_CTR, nonce=b'')
        encrypted = aes.encrypt(flag.encode()).hex()
        print(f'flag:', encrypted)
        return (publicA,verA,encrypted)
    else:
        print(f'nope! {shared:#x}')
        return (publicA,verA,shared)

In [9]:
from Crypto.Util.number import isPrime
import math
assert isPrime(q)
math.log(q)

528.080102389484

In [27]:
from sympy import factorint

factorint(q-1)
# => No generator based attack i.e. small subgroups

{2: 2,
 2344807743588794553832391292516279194397209456764712786969868894104465782493871625440983981162219279755855675661203: 2}

In general, I was to lazy to build the oracle with pwntools, therefore I copy and pasted in the values

## Step 1)

Get the generator by sending $q-1$ as our public key. This will lead to $(q-1)^{sk} \mod q$ where $sk$ is even, such that the shared secret is $1$.

In [18]:
publicB = q-1 

publicA,verA,shared = go(publicB,-1)

assert shared == 1

"Send payload:",publicB

publicA = 0x189e8ba7966eb29cbf73f866221382ba5286038015a2876917be756c04c08c36511f10c81276b2ba4bdbbc15b7fb3d88795eda71da3ca558a11c1afe91f59538034d0a3e84ba7c0db4da8225ed9eb95a2d3c113f71ae00ea47a02a639326d27
verA = 0x92ad7d86a66dd2c1877006724f89c4747987875bb9e1a607499e767a9e35746e
nope! 0x1


('Send payload:',
 21992493417575896428286087521674334179336251497851906051131955410904158485314789427947788692030188502157019527331790513011401920585195969087140918256569620608732530453375717414098148438918130733211117668960801178110820764957628836)

In [20]:
# pastes in verA
generator = int("0xc36896ad2a0a7e4b9002b57d41fad70bfc67017835c9139b0486fbee7a773a22",16)
generator

88385797498924122664683714702432294121015094809694995437885709393914970454562

## Step 2

Now we have to retrieve the encrypted flag, therefore we have to solve the small challenge of computing $g^{(shared^{5})}$. Therefore, we can send $Pk_b = q-1$ again, but this time have to set $V_b = g$

In [28]:
publicB = q-1 

publicA,verA,ciphertext = go(publicB,generator)

("send following",q-1,generator)

publicA = 0x29257af8795df9d386ce1ca378b601a89988a847c0f3c76b10ee65c738328116e494b287f8b0550ac052aeec81ad0961b48550370a7c509dd0b8d332a0ef0d1f5dad22d1c01a8fbde09197aa70d3bae3c6a38cd9e7cb9242f0630687f2b775c
verA = 0x92ad7d86a66dd2c1877006724f89c4747987875bb9e1a607499e767a9e35746e
nope! 0x1


('send following',
 21992493417575896428286087521674334179336251497851906051131955410904158485314789427947788692030188502157019527331790513011401920585195969087140918256569620608732530453375717414098148438918130733211117668960801178110820764957628836,
 88385797498924122664683714702432294121015094809694995437885709393914970454562)

In [29]:
# paste the ciphertext from the oracle
ciphertext = bytes.fromhex("f4cb9c2e2ccac7124c4b86041adf731d3c2b22a166f819403c05ad75fa28480c3435b99beb03c19e")

## Step 3

Bruteforce the password because there are only $10^6=1.000.000$ possibilities 

In [23]:
flags = [] 

import string
pable = set(bytes(string.printable.encode()))

for i in range(10**6):
    key = H(i,1)
    aes = AES.new(key, AES.MODE_CTR, nonce=b'')

    plaintext = aes.decrypt(ciphertext)

    # check if chars are printable
    if set(plaintext).issubset(pable):
        flags.append(plaintext)

list(filter(lambda x:b"{" in x,flags))